# Movie Recommender System

## Types
1. Content-based Recommendation System
2. Popularity-based Recommendation System
3. Collaborative Recommendation System

## Data

## Data Pre-processing

## Feature Extraction

## User Input

## Cosine Similarity Algorithm